In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.preprocessing import StandardScaler
import pandas as pd
import mysql.connector
from mysql.connector import Error
import numpy as np
from sklearn.model_selection import train_test_split


#### CONNECT TO GOOGLE CLOUD MYSQL
config = {
    'user': 'root',
    'password': 'txybkx5kPMpko8C6',
    'host': '35.238.14.103', 
    'database': 'ddx_db'
}
cnxn = mysql.connector.connect(**config)

#### BASIC QUERY FUNCTIONS

# Read query, something like a select
def read_query(connection, query):
    cursor = connection.cursor()
    result = None
    try:
        cursor.execute(query)
        result = cursor.fetchall()
        return result
    except Error as err:
        print(f"Error: '{err}'")



In [6]:
def merged_nba_game_data_df():
    # Query NBA Game Data to get dataset
    q1 = """
    SELECT * FROM nba_game_data;
    """

    nba_game_data = read_query(cnxn, q1)
    #print(nba_game_data)
    cols = box_stats = ['id', 'gid', 'team', 'fg', 'fga', 'fg_pct', 'fg3', 'fg3a', 'fg3_pct', 'ft', 'fta', 'ft_pct', 'orb', 'drb', 'trb', 'ast', 'stl', 'blk', 'tov', 'pf', 'pts', 'ts_pct', 'efg_pct', 'fg3a_per_fga_pct', 'fta_per_fga_pct', 'orb_pct', 'drb_pct', 'trb_pct', 'ast_pct', 'stl_pct', 'blk_pct', 'tov_pct', 'off_rtg', 'def_rtg', 'date']
    ngd_df = pd.DataFrame(nba_game_data, columns=cols)


    #print(ngd_df.team)

    df_full_game_join = ngd_df.join(ngd_df.shift(-1).add_prefix('away_'))
    df_full_game_join[1::2] = ''
    df_full_game_join = df_full_game_join[df_full_game_join.gid != '']
    cols1 = ['team', 'away_team', 'pts', 'away_pts']
    df_full_game_join['key'] = df_full_game_join[cols1].apply(lambda row: '_'.join(row.values.astype(str)), axis=1)
    df_full_game_join['key'] = df_full_game_join['key'].map(lambda x: str(x)[:-2])
    return df_full_game_join

# Create average of all df for all teams
def avg_nba_game_data_df():
    df = merged_nba_game_data_df()
    stat_cols = cols = ['fg', 'fga', 'fg_pct', 'fg3', 'fg3a', 'fg3_pct', 'ft', 'fta',
       'ft_pct', 'orb', 'drb', 'trb', 'ast', 'stl', 'blk', 'tov', 'pf', 'pts',
       'ts_pct', 'efg_pct', 'fg3a_per_fga_pct', 'fta_per_fga_pct', 'orb_pct',
       'drb_pct', 'trb_pct', 'ast_pct', 'stl_pct', 'blk_pct', 'tov_pct',
       'off_rtg', 'def_rtg', 'away_fg', 'away_fga', 'away_fg_pct', 'away_fg3',
       'away_fg3a', 'away_fg3_pct', 'away_ft', 'away_fta', 'away_ft_pct',
       'away_orb', 'away_drb', 'away_trb', 'away_ast', 'away_stl', 'away_blk',
       'away_tov', 'away_pf', 'away_pts', 'away_ts_pct', 'away_efg_pct',
       'away_fg3a_per_fga_pct', 'away_fta_per_fga_pct', 'away_orb_pct',
       'away_drb_pct', 'away_trb_pct', 'away_ast_pct', 'away_stl_pct',
       'away_blk_pct', 'away_tov_pct', 'away_off_rtg', 'away_def_rtg']
    
    df = df.drop(columns=['id', 'gid', 'date', 'key', 'away_team', 'away_date', 'away_id', 'away_gid'])
    df[stat_cols] = df[stat_cols].apply(pd.to_numeric)
    df = df.groupby('team', axis=0).mean().reset_index() 
    return df




In [14]:
all_nba = merged_nba_game_data_df()
all_nba = all_nba[['team', 'away_team', 'pts', 'away_pts']]

team_avgs_df = avg_nba_game_data_df()
print(team_avgs_df.head())
print(all_nba.head())

# making win loss column
all_nba['Win_Loss'] = np.where(all_nba['pts'] > all_nba['away_pts'], 1.0, 0.0)

y = all_nba['Win_Loss']

X = all_nba.merge(team_avgs_df, how='left', on='team').merge(team_avgs_df, how='left', left_on='away_team', right_on='team')

# print all cols
pd.set_option('display.max_columns', None)
print(X['away_def_rtg_y'])

  team         fg        fga    fg_pct        fg3       fg3a   fg3_pct  \
0  atl  39.718750  87.718750  0.454344  12.812500  36.531250  0.345688   
1  bos  41.187500  88.468750  0.467281  12.000000  32.031250  0.378469   
2  brk  45.068966  89.241379  0.505138  15.551724  38.000000  0.410552   
3  chi  44.428571  89.964286  0.494893  13.357143  34.142857  0.386821   
4  cho  41.672131  91.065574  0.458000  17.459016  38.918033  0.445607   

          ft        fta    ft_pct        orb        drb        trb        ast  \
0  21.156250  25.562500  0.826312  11.343750  34.562500  45.906250  23.218750   
1  16.500000  22.031250  0.746250  10.906250  32.687500  43.593750  21.843750   
2  17.862069  22.103448  0.808655   8.862069  33.724138  42.586207  27.482759   
3  15.964286  20.142857  0.803071   8.928571  36.392857  45.321429  27.642857   
4  19.688525  24.442623  0.796033  11.868852  31.409836  43.278689  29.327869   

        stl       blk        tov         pf         pts    ts_pct   

In [21]:
# Predicitng game score with jus offense defense of ratings 
X = X[['off_rtg_x', 'def_rtg_x', 'off_rtg_y', 'def_rtg_y', 'away_off_rtg_x', 'away_def_rtg_x', 'away_off_rtg_y', 'away_def_rtg_y']]
y = all_nba[['pts', 'away_pts']].values

X_train, X_test, y_train, y_test = train_test_split(X, y)


parameters = {'bootstrap': False,
              'min_samples_leaf': 3,
              'n_estimators': 50,
              'min_samples_split': 10,
              'max_features': 'sqrt',
              'max_depth': 6}
model = RandomForestRegressor(**parameters)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)



print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))





Mean Absolute Error: 7.486449868769453
Mean Squared Error: 105.90667244720089
Root Mean Squared Error: 10.29109675628409
